In [1]:
#Installing the dependencies
def what_is_installed():
    import pycaret
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except:
    !pip install prophet
    !pip install pycaret-ts-alpha
    what_is_installed()


System:
    python: 3.9.7 (default, Sep 16 2021, 08:50:36)  [Clang 10.0.0 ]
executable: /Users/sage/opt/anaconda3/bin/python
   machine: macOS-10.16-x86_64-i386-64bit

PyCaret required dependencies:
                 pip: 22.3.1
          setuptools: 58.0.4
             pycaret: 3.0.0rc4
             IPython: 7.29.0
          ipywidgets: 7.6.5
                tqdm: 4.62.3
               numpy: 1.21.6
              pandas: 1.4.4
              jinja2: 3.1.2
               scipy: 1.7.3
              joblib: 1.2.0
             sklearn: 1.0.2
                pyod: 1.0.5
            imblearn: 0.9.0
   category_encoders: 2.5.1.post0
            lightgbm: 3.3.2
               numba: 0.55.2
            requests: 2.28.1
          matplotlib: 3.4.3
          scikitplot: 0.3.7
         yellowbrick: 1.4
              plotly: 5.5.0
             kaleido: 0.2.1
         statsmodels: 0.13.2
              sktime: 0.13.4
               tbats: 1.1.1
            pmdarima: 1.8.5
              psutil: 5.9.2


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import plotly.express as px
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from numpy import log

In [3]:
import os
import sys
import datetime as dt
#import missingno as mno 

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [4]:
# load dataset
import pandas as pd
may = pd.read_csv("Measurement May 2022.csv")
june = pd.read_csv("Measurement june 2022.csv")
july = pd.read_csv("Measurements July 2022.csv")
august = pd.read_csv("Measurement August 2022.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Measurement May 2022.csv'

In [ ]:
#checking some rows of the May dataset
may.head()

In [ ]:
#concatenate
df = pd.concat([may,june,july,august])
df.head()

In [ ]:
#Tail of dataframe
df.tail()

In [ ]:
#correcting the index
df=df.reset_index(drop=True)

In [ ]:
#checking the tail
df.tail()

In [ ]:
#changing the Metres above MSL and TOW MH20 no-data values to null
df["Metres above MSL"] = df["Metres above MSL"].replace(["no-data"],0)
df["TOW _MH2O"] = df["TOW _MH2O"].replace(["no-data"],0)

In [ ]:
#checking the data after replacing some columns with null values
df.head()

In [ ]:
# concatenating the dates
df['data_date'] = df['data_date'].astype(str)
df['data_time'] = df['data_time'].astype(str)
df['DateTime'] = df['data_date'] + ' ' + df['data_time']
df.info()

In [ ]:
# changing to date
df['data_date'] = pd.to_datetime(df['data_date'], format = '%d/%m/%Y')
df.head()

In [ ]:
#changing the datetime to the right format
df['DateTime'] = pd.to_datetime(df['DateTime'], format = '%d/%m/%Y %H:%M:%S')
df.head()

In [ ]:
#checking the info
df.info()

In [ ]:
#copy into level_1
level_1 = df.copy()
level_1.shape

In [ ]:
#Level 1 10-30, dropping the other columns and date
level_1.drop(level_1.columns[[10,12,13,14,15,16,17,34,35,36]], axis =1, inplace = True)
level_1

In [ ]:
#the shape of the level_1
level_1.shape

In [ ]:
# Removing duplicates
new = level_1.drop_duplicates()

In [ ]:
# checking the shape
new.shape

In [ ]:
#checking the info
new.info()

In [ ]:
#Installing Boruta
!pip install Boruta

In [ ]:
#Installing Boruta
!pip install Boruta
# Boruta for feature selection
!pip install boruta_py
from sklearn.ensemble import RandomForestRegressor
from boruta import  BorutaPy
from sklearn.metrics import accuracy_score 

In [ ]:
X_new = new.drop(columns =["Magna_6 Segment 1_ (10-30cm) Soil Moisture_%"])

In [ ]:
X_new.shape

In [ ]:
X_new.info()

In [ ]:
Y_new = new["Magna_6 Segment 1_ (10-30cm) Soil Moisture_%"]

In [ ]:
Y_new

In [ ]:
#Y variable
Y_new.head()

In [ ]:
#Splitting the dataset.
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_new, test_size= 0.2, random_state= 1)

In [ ]:
#Initializing the random forest model
model = RandomForestRegressor(n_estimators = 100, max_depth = 5, random_state = 42)

In [ ]:
# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

In [ ]:
#Training the boruta, X and Y mus be numpy arrays
import numpy as np
feat_selector.fit(np.array(X_new), np.array(Y_new))


In [ ]:
# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", X_new.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              X_new.columns[i], " - Ranking: ", feat_selector.ranking_[i])

In [ ]:
#checking the dataset to remove the unimportant features.
new.info()

In [ ]:
#Getting the  leve_One only
level_One = new.copy()
level_One.shape

In [ ]:
#Level 1 10-30, dropping the other columns and date
segment1 = level_One.iloc[:,[3,7,8,9,10,11,13,17,21,22,23,24,25,26]]
segment1

In [ ]:
#checking the important variables
segment1.info()

In [ ]:
#Adding the date colum to segment1
DateTime = df["DateTime"]

In [ ]:
segment1["DateTime"] = DateTime

In [ ]:
segment1.info()

In [ ]:
#changing column 12 and 13 to float
segment1['Metres above MSL'] = pd.to_numeric(df['Metres above MSL'])
segment1['TOW _MH2O'] = pd.to_numeric(df['TOW _MH2O'])

In [ ]:
segment1.info()

In [ ]:
#setting an index
segment1.set_index("DateTime", inplace = True)

In [ ]:
#Ploting the dataset
segment1.plot(subplots = True, figsize = (12,65));

In [ ]:
# future dataset
future_data = segment1.iloc[-7000:]
future_data

In [ ]:
from pycaret.time_series import *

# Change renderer appropriately based on where the notebook is being run ----
# Refer to plotly for available renderers.
global_plot_settings = {"renderer": "colab",
                        "hoverinfo": "text"}
# init environment
from pycaret.regression import *
exp = TSForecastingExperiment()
target = 'Magna_6 Segment 1_ (10-30cm) Soil Moisture_%'
data_for_modeling = segment1[segment1.index < '2022-08-21'].reset_index(drop = True)
future_df = segment1[segment1.index >= '2022-08-21'].reset_index(drop = True)
future_exog = future_df.drop(columns=target)

exp.setup(data = data_for_modeling,
          target= target,
          seasonal_period=12,
          fh=1000,
          fig_kwargs=global_plot_settings,
          session_id=123)


In [ ]:
# Additional plots provided by PyCaret to help understand what is being done.
exp.plot_model(plot="train_test_split")

In [ ]:
#Model training and selection
print("Model training and selection")
best_model = exp.compare_models(sort = 'MAE')

In [ ]:
exp.plot_model(best_model,
               plot = 'forecast')

In [ ]:
tuned= exp.tune_model(best_model,fold=5)

In [ ]:
# finalize model
final_best = exp.finalize_model(tuned)
print(final_best)

In [ ]:
# generate predictions
preds= exp.predict_model(final_best,fh=1716,X=future_exog)
print(preds)

In [ ]:
y_train= data_for_modeling['segment1(10-30cm)']
#y_train=exp.get_config("y_train")
y_train
print(y_train)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,preds)